## <em> Needed tools importation </em>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#basics tools
import matplotlib.pyplot as plt
import numpy as np
import  pandas as pd
#tensorflow functionnality
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers,models

## <em>Loading data</em>

## <em> tf_tensor helps us to read images by batch and pretty useful for apply stuff by group shuffle parameter importante,it's help us to randomize the images you may notice when your executing the code more times images changes at the same times</em>

In [3]:
IMG_SIZE=256
BATCH_SIZE=32
CHANNEL=3
EPOCHS=50

In [7]:
datasets=tf.keras.preprocessing.image_dataset_from_directory(
"/content/drive/MyDrive/googleColabStuff/ColabStuff/Deep-learning-stuff/myProject/patatoes-datasets",
shuffle=True,
image_size=(IMG_SIZE,IMG_SIZE),
batch_size=BATCH_SIZE
)

FileNotFoundError: ignored

In [ ]:
className=datasets.class_names
className

In [ ]:
rows=columns=4

for img_batch,label_batch in datasets.take(1):
   fig=plt.figure(figsize=(10,10))
   for index in range(12):
      fig.add_subplot(rows,columns,index+1)
      # showing image
      plt.imshow(img_batch[index].numpy().astype("uint8"))
      plt.axis('off')
      plt.title(f'{className[label_batch[index].numpy()]}')
      

##  <em>Splitting data </em>

In [ ]:
def split_datasets_tf(ds,train_split=.8,test_split=.1,val_split=.1,shuffled=True,shuffle_size=10000):
    ds_size=len(ds)
    if shuffled:
        ds.shuffle(shuffle_size,seed=111)
    train_size=int(ds_size*train_split)
    val_size=int(ds_size*val_split)      # we will take the same for test size
       
    train_ds= ds.take(train_size)
    remain_ds=ds.skip(train_size)
    val_ds=remain_ds.take(val_size)
    test_ds=remain_ds.skip(val_size)
        
    return train_ds,val_ds,test_ds

In [ ]:
train_ds,val_ds,test_ds=split_datasets_tf(datasets)
print('train_ds size:',len(train_ds))
print('test_ds size:',len(test_ds))
print('val_ds size:',len(val_ds))

## <em> Useful for speeding up the training processs </em>

In [ ]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

## <em> data preprocessing </em> 

In [ ]:
data_resize_recale=tf.keras.Sequential([
 layers.experimental.preprocessing.Resizing(IMG_SIZE,IMG_SIZE),
 layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [ ]:
data_augmentation=keras.Sequential([
    layers.experimental.preprocessing.RandomContrast(.1),
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(.3),
    layers.experimental.preprocessing.RandomZoom(.2)
])

In [ ]:
INPUT_SHAPE=(BATCH_SIZE,IMG_SIZE,IMG_SIZE,CHANNEL)
model=keras.Sequential([
   #feature extraction
    data_resize_recale,
    data_augmentation,
    layers.Conv2D(256,(3,3),activation='relu',input_shape=INPUT_SHAPE),
    layers.MaxPool2D((2,2)),
    
    keras.layers.Conv2D(128,3,strides=(1,1),padding='same',activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    layers.Dropout(.2),
    
    keras.layers.Conv2D(64,3,strides=(1,1),padding='same',activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    layers.Dropout(.1),
    
    #feature classification
    layers.Flatten(),
    keras.layers.Dense(32,activation='relu'),
    keras.layers.Dense(5,activation='softmax')
    
])
model.build(input_shape=INPUT_SHAPE)

In [ ]:
model.summary()

In [ ]:
model.compile(
optimizer='adam',
loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
metrics=['accuracy']
)

In [ ]:
histo=model.fit(
train_ds,
epochs=EPOCHS,
batch_size=BATCH_SIZE,
verbose=1,
validation_data=val_ds
)

In [ ]:
score=model.evaluate(test_ds)

In [ ]:
score

In [ ]:
histo.params

In [ ]:
histo.history.keys()

In [ ]:
len(histo.history['accuracy'])

In [ ]:
acc_tab=histo.history['accuracy']
val_tab=histo.history['val_accuracy']
val_loss=histo.history['val_loss']
loss=histo.history['loss']

<em> learning map </em>

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc_tab,label='training accuracy')
plt.plot(range(EPOCHS),val_tab,label='validation accuracy')
plt.legend(loc='lower right')
plt.title('traing and validation accuracy')

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label='training loss')
plt.plot(range(EPOCHS),val_loss,label='validation loss')
plt.legend(loc='upper right')
plt.title('traing and validation loss')

<em> fewer predictions </em>

In [ ]:
for img_batch,labels_batch in test_ds.take(1):
    first_img=img_batch[0].numpy().astype('uint8')
    actual_label=label_batch[0].numpy()
    
    print('First image to predict')
    print('actual label:',className[actual_label])
    plt.imshow(first_img)
    plt.axis('off')

    predicted_batch=model.predict(img_batch)

    print('predicted label:',className[np.argmax(predicted_batch[0])])

<em> predicted function </em>

In [ ]:
def my_predicted_function(model,image):
    img_array=tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array=tf.expand_dims(img_array,0) #creat a batch
    
    predictions=model.predict(img_array)

    predicted_class=className[np.argmax(predictions[0])]
    confidency=round(100*(np.max(predictions[0])),2)
    return predicted_class,confidency

In [ ]:
rows=columns=4

for images,labels in test_ds.take(1):
   fig=plt.figure(figsize=(18,18))
   for index in range(12):
      fig.add_subplot(rows,columns,index+1)
      predicted_class,confidency=my_predicted_function(model,images[i].numpy())
      # showing image
      plt.imshow(images[index].numpy().astype("uint8"))
      plt.axis('off')
      plt.title(f'actual labels:{className[labels[index].numpy()]} \n predicted label:{predicted_class} \n confidency:{confidency}')